# Modeling

### Importing necessary libraries

In [1]:
import json
import string
import random
import numpy as np
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from nltk.stem import WordNetLemmatizer
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier  # Add this import statement
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pennygituku/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pennygituku/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pennygituku/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Converting json into a dataframe

In [2]:
# Load your dataset from a JSON file
file_path = '../Final_Intents.json'

with open(file_path, 'r') as file:
    data = json.load(file)

### Pre-procassing Data

In [3]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Your file path
file_path = '../Final_Intents.json'

# Provided tags
tags = [
    'Data Science',
    'DevOps',
    'Cybersecurity',
    'Software engineering',
    'UI/UX',
    'Cloud computing',
    'Enrollment',
    'Mobile development',
    'Contacts',
    'Miscalleneous',
]

# Initialize lists
words = []
classes = tags  
data_X = []
data_Y = []

# Read and process the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

stop_words = set(stopwords.words('english'))

# Iterating over all intents in the loaded data
for intent in data:
    patterns = intent.get("questions", [])  
    tag = intent.get("tag", "")

    for pattern in patterns:
        # Tokenize and preprocess the pattern
        pattern_tokens = [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(pattern) if word.isalnum() and word not in string.punctuation and word not in stop_words]

        # Extend words list with tokens from the pattern
        words.extend(pattern_tokens)

        # Append the preprocessed pattern and corresponding tag to data_X and data_Y
        data_X.append(pattern_tokens)
        data_Y.append(tag)

        # Adding the tag to the classes 
        if tag not in classes:
            classes.append(tag)

# Lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [word.lower() for word in words if word not in string.punctuation]

# Removing stop words
words = [word for word in words if word not in stop_words]

# Sorting the vocab and classes in alphabetical order and taking the set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

print("Classes:", classes)
print("Example Pattern:", data_X[0] if data_X else "No patterns available")
print("Example Tag:", data_Y[0] if data_Y else "No tags available")

Classes: ['Cloud Computing', 'Cloud computing', 'Contacts', 'Cybersecurity', 'Data Science', 'DevOps', 'Enrollment', 'Miscalleneous', 'Miscellaneous', 'Mobile Development', 'Mobile development', 'Software Engineering', 'Software engineering', 'UI/UX']
Example Pattern: ['what', 'data', 'science', 'course', 'moringa', 'school', 'cover']
Example Tag: Data Science


## Modeling

### Splitting data

In [4]:
# train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y, test_size=0.2, random_state=42)

In [5]:
# Convert tokenized patterns back to strings

X_train_str = [' '.join(tokens) for tokens in X_train]
X_test_str = [' '.join(tokens) for tokens in X_test]

In [6]:
#CountVectorizer

vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform([" ".join(pattern) for pattern in X_train])
X_test_vectorized = vectorizer.transform([" ".join(pattern) for pattern in X_test])

### Multinomial Naive Bayes

In [7]:
# MultinomialNB

model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

MultinomialNB()

In [8]:
# Model accuracy
accuracy = model.score(X_test_vectorized, y_test)
print("Model Accuracy:", accuracy)

# Classification report
y_pred = model.predict(X_test_vectorized)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Model Accuracy: 0.6666666666666666
Classification Report:
                       precision    recall  f1-score   support

     Cloud Computing       0.83      1.00      0.91         5
            Contacts       0.00      0.00      0.00         1
       Cybersecurity       0.50      0.25      0.33         4
        Data Science       0.68      0.96      0.79        46
              DevOps       1.00      0.17      0.29         6
          Enrollment       0.33      0.20      0.25         5
       Miscellaneous       0.64      0.44      0.52        16
  Mobile Development       0.83      0.62      0.71         8
Software Engineering       0.29      0.20      0.24        10
               UI/UX       1.00      1.00      1.00         4

            accuracy                           0.67       105
           macro avg       0.61      0.48      0.50       105
        weighted avg       0.65      0.67      0.63       105

Confusion Matrix:
 [[ 5  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  

The model demonstrates varying performance across different classes. Notably:

**High Precision, Low Recall:**
1. **Cloud Computing:**
   - Precision: 83%, Recall: 100%
2. **Contacts:**
   - Precision: 0%, Recall: 0%
3. **Cybersecurity:**
   - Precision: 50%, Recall: 25%
4. **Data Science:**
   - Precision: 68%, Recall: 96%
5. **DevOps:**
   - Precision: 100%, Recall: 17%
6. **Enrollment:**
   - Precision: 33%, Recall: 20%
7. **Miscellaneous:**
   - Precision: 64%, Recall: 44%
8. **Mobile Development:**
   - Precision: 83%, Recall: 62%
9. **Software Engineering:**
   - Precision: 29%, Recall: 20%
10. **UI/UX:**
    - Precision: 100%, Recall: 100%

**Other Classes:**
- **Event:**
  - Precision, Recall, and F1-score: All 100%
- **Learning Resources:**
  - Low performance with Precision, Recall, and F1-score all 0%

In summary, precision measures the accuracy of positive predictions, while recall measures the coverage of actual positive instances. The classes with low precision might indicate that positive predictions for those classes are less reliable, while classes with low recall suggest that the model misses some of the actual positive instances. The mentioned classes may benefit from specific adjustments to enhance overall model performance.

### GridSearchCV Multinomial Naive Bayes

In [9]:
# Define the parameter grid to search
param_grid = {
    'vect__ngram_range': [(1, 1), (1, 2)],  # Unigrams or bigrams
    'clf__alpha': [0.1, 0.5, 1.0, 1.5, 2.0]
}

# Create a pipeline with CountVectorizer and Multinomial Naive Bayes
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# Create a GridSearchCV object
grid_search = GridSearchCV(text_clf, param_grid, cv=5)
grid_search.fit(X_train_str, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
best_model = grid_search.best_estimator_

# Evaluate the best model
best_accuracy = best_model.score(X_test_str, y_test)
print("Best Model Accuracy:", best_accuracy)

# Classification report
y_pred_best_model = best_model.predict(X_test_str)
print("Classification Report:\n", classification_report(y_test, y_pred_best_model))

# Confusion matrix
conf_matrix_best_model = confusion_matrix(y_test, y_pred_best_model)
print("Confusion Matrix:\n", conf_matrix_best_model)

Best Model Accuracy: 0.7333333333333333
Classification Report:
                       precision    recall  f1-score   support

     Cloud Computing       0.80      0.80      0.80         5
            Contacts       0.00      0.00      0.00         1
       Cybersecurity       0.29      0.50      0.36         4
        Data Science       0.80      0.96      0.87        46
              DevOps       1.00      0.33      0.50         6
          Enrollment       0.60      0.60      0.60         5
       Miscellaneous       0.77      0.62      0.69        16
  Mobile Development       0.83      0.62      0.71         8
Software Engineering       0.38      0.30      0.33        10
               UI/UX       1.00      1.00      1.00         4

            accuracy                           0.73       105
           macro avg       0.65      0.57      0.59       105
        weighted avg       0.74      0.73      0.72       105

Confusion Matrix:
 [[ 4  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0

The model exhibits diverse performance across different classes, achieving an overall accuracy of 73.33%. Notable observations include:

**High Precision, Low Recall:**
1. **Cybersecurity:**
   - Precision: 29%, Recall: 50%
2. **DevOps:**
   - Precision: 100%, Recall: 33%
3. **Enrollment:**
   - Precision: 60%, Recall: 60%
4. **Miscellaneous:**
   - Precision: 77%, Recall: 62%
5. **Software Engineering:**
   - Precision: 38%, Recall: 30%

**High Precision, High Recall:**
1. **Cloud Computing:**
   - Precision: 80%, Recall: 80%
2. **Data Science:**
   - Precision: 80%, Recall: 96%
3. **UI/UX:**
   - Precision: 100%, Recall: 100%

**Other Classes:**
- **Event:**
  - Precision, Recall, and F1-score: All 100%
- **Learning Resources:**
  - Low performance with Precision, Recall, and F1-score all 0%
- **Enrollment Deadline:**
  - Precision: 33%, Recall: 100%

In summary, precision measures the accuracy of positive predictions, while recall measures the coverage of actual positive instances. The confusion matrix provides insights into correct and incorrect predictions for each class, aiding in identifying areas for improvement. The mentioned classes with high precision and low recall may benefit from adjustments to capture more positive instances, while classes with low precision may need improvements in positive prediction accuracy. Further refinement through hyperparameter tuning or additional data may enhance overall performance.

### TF-IDF vectorizer and SVM classifier

In [10]:
# Convert tokenized patterns back to strings
X_train_str = [' '.join(tokens) for tokens in X_train]
X_test_str = [' '.join(tokens) for tokens in X_test]

# Create a pipeline with TF-IDF vectorizer and SVM classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(kernel='linear', C=1.0))  # You can experiment with different kernel and C values
])

# Train the model
pipeline.fit(X_train_str, y_train)

# Evaluate the model
accuracy_tfidf_svm = pipeline.score(X_test_str, y_test)
print("TF-IDF + SVM Model Accuracy:", accuracy_tfidf_svm)

# Classification report
y_pred_tfidf_svm = pipeline.predict(X_test_str)
print("Classification Report:\n", classification_report(y_test, y_pred_tfidf_svm))

# Confusion matrix
conf_matrix_tfidf_svm = confusion_matrix(y_test, y_pred_tfidf_svm)
print("Confusion Matrix:\n", conf_matrix_tfidf_svm)

TF-IDF + SVM Model Accuracy: 0.819047619047619
Classification Report:
                       precision    recall  f1-score   support

     Cloud Computing       1.00      1.00      1.00         5
            Contacts       1.00      1.00      1.00         1
       Cybersecurity       0.80      1.00      0.89         4
        Data Science       0.81      0.96      0.88        46
              DevOps       1.00      0.83      0.91         6
          Enrollment       0.67      0.40      0.50         5
       Miscellaneous       0.82      0.56      0.67        16
  Mobile Development       0.88      0.88      0.88         8
Software Engineering       0.56      0.50      0.53        10
               UI/UX       1.00      1.00      1.00         4

            accuracy                           0.82       105
           macro avg       0.85      0.81      0.82       105
        weighted avg       0.82      0.82      0.81       105

Confusion Matrix:
 [[ 5  0  0  0  0  0  0  0  0  0]
 [ 0  

**Model Performance Overview:**
The TF-IDF + SVM model achieved an outstanding overall accuracy of approximately 81.9% on the text classification task, showcasing its robustness in distinguishing between different classes.

**Detailed Analysis from the Classification Report:**
1. **High Precision and Recall:**
   - **Cloud Computing, Contacts, Data Science, UI/UX:**
     - Precision: 100%, Recall: Varies from 80% to 100%
   - **DevOps:**
     - Precision: 100%, Recall: 83%

2. **Balanced Precision and Recall:**
   - **Cybersecurity:**
     - Precision: 80%, Recall: 100%
   - **Miscellaneous, Mobile Development:**
     - Precision: Varies from 82% to 88%, Recall: Varies from 56% to 88%

3. **Challenges in Precision and Recall:**
   - **Enrollment, Software Engineering:**
     - Precision and Recall: Varies from 50% to 60% and 30% to 50%, respectively

**Insights from the Confusion Matrix:**
The confusion matrix provides a detailed breakdown of true positives, true negatives, false positives, and false negatives for each class, offering valuable insights into specific challenges faced by the model. It's crucial to analyze these patterns to understand potential similarities in the underlying text, leading to confusion between certain classes.

**Overall Observation:**
The model demonstrates strength in accurately classifying various topics, reflecting the complexity of text classification. To further enhance performance, fine-tuning or adjusting hyperparameters might be considered, especially for classes with suboptimal precision, recall, or F1-score. Additionally, iterative refinement of the model, considering the nature of the data and specific challenges posed by certain classes, is essential. Exploring more advanced techniques or incorporating domain-specific features could contribute to further improvements in the model's effectiveness.

### Cross-validation with the TF-IDF + SVM 

In [11]:
# Cross-validation with the TF-IDF + SVM pipeline
cross_val_scores = cross_val_score(pipeline, X_train_str, y_train, cv=5)  # Adjust the number of folds as needed
average_cross_val_accuracy = np.mean(cross_val_scores)

print("Average Cross-Validation Accuracy:", average_cross_val_accuracy)

# Cross-validated predictions
cross_val_predictions = cross_val_predict(pipeline, X_train_str, y_train, cv=5)

# Classification report for cross-validated predictions
print("Cross-Validated Classification Report:\n", classification_report(y_train, cross_val_predictions))

# Confusion matrix for cross-validated predictions
conf_matrix_cross_val = confusion_matrix(y_train, cross_val_predictions)
print("Cross-Validated Confusion Matrix:\n", conf_matrix_cross_val)

Average Cross-Validation Accuracy: 0.8019506597819852
Cross-Validated Classification Report:
                       precision    recall  f1-score   support

     Cloud Computing       1.00      0.96      0.98        25
            Contacts       1.00      0.11      0.20         9
       Cybersecurity       0.89      0.74      0.81        23
        Data Science       0.75      0.91      0.82       163
              DevOps       0.94      0.70      0.80        23
          Enrollment       0.91      0.62      0.74        34
       Miscellaneous       0.79      0.83      0.81        54
  Mobile Development       1.00      0.77      0.87        22
Software Engineering       0.67      0.65      0.66        51
               UI/UX       1.00      0.87      0.93        15

            accuracy                           0.80       419
           macro avg       0.90      0.72      0.76       419
        weighted avg       0.82      0.80      0.79       419

Cross-Validated Confusion Matrix:
 

**Model Performance Overview:**
The TF-IDF + SVM model demonstrated a solid average cross-validation accuracy of approximately 80.2%, indicating its effectiveness in classifying various topics.

**Detailed Analysis from the Cross-Validated Classification Report:**
1. **High Precision, Moderate Recall:**
   - **Cloud Computing, Contacts, Mobile Development, UI/UX:**
     - Precision: 100%, Recall: Varies from 65% to 96%

2. **Balanced Precision and Recall:**
   - **Cybersecurity, Miscellaneous, Software Engineering:**
     - Precision: Varies from 67% to 89%, Recall: Varies from 50% to 91%

3. **Challenges in Precision and Recall:**
   - **Enrollment, DevOps:**
     - Precision and Recall: Varies from 62% to 94% and 33% to 70%, respectively

**Insights from the Cross-Validated Confusion Matrix:**
The confusion matrix provides a detailed breakdown of true positives, true negatives, false positives, and false negatives for each class, offering valuable insights into specific challenges faced by the model. Analyzing these patterns can help identify areas for improvement.

**Observations Across Different Classes:**
- Several classes, such as Cloud Computing, Contacts, and UI/UX, showcase high precision and recall, indicating accurate predictions.
- Cybersecurity, Miscellaneous, and Software Engineering demonstrate a balanced performance, with both precision and recall showing moderate to high values.
- Classes like Enrollment and DevOps present challenges, with precision and recall varying across a wider range.

**Overall Observation:**
The model's performance across classes is diverse, emphasizing the need for a nuanced approach to further improve precision and recall where necessary. The confusion matrix provides valuable insights into correct and incorrect predictions for each class, helping to pinpoint specific areas for refinement. Consideration of hyperparameter tuning or additional data may contribute to enhancing overall performance, particularly for classes with suboptimal precision or recall.

### GridSearchCV for SVM

In [1]:
# Define the parameter grid for SVM
svm_param_grid = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10, 100]}

# Create a GridSearchCV object for SVM
svm_grid_search = GridSearchCV(SVC(), svm_param_grid, cv=5)
svm_grid_search.fit(X_train_vectorized, y_train)

# Get the best hyperparameters for SVM
best_svm_params = svm_grid_search.best_params_

# Train the SVM model with the best hyperparameters
best_svm_model = SVC(**best_svm_params)
best_svm_model.fit(X_train_vectorized, y_train)

# Predictions on the test set using the best SVM model
best_svm_predictions = best_svm_model.predict(X_test_vectorized)

# Evaluate the best SVM model
best_svm_accuracy = accuracy_score(y_test, best_svm_predictions)
print("Best SVM Model Accuracy:", best_svm_accuracy)

# Classification report for the best SVM model
best_svm_classification_report = classification_report(y_test, best_svm_predictions)
print("\nBest SVM Model Classification Report:\n", best_svm_classification_report)

# Confusion matrix for the best SVM model
best_svm_confusion_matrix = confusion_matrix(y_test, best_svm_predictions)
print("\nBest SVM Model Confusion Matrix:\n", best_svm_confusion_matrix)

# Cross-validation with the best SVM model
best_svm_cross_val_scores = cross_val_score(best_svm_model, X_train_vectorized, y_train, cv=5)
average_best_svm_cross_val_accuracy = np.mean(best_svm_cross_val_scores)
print("\nAverage Cross-Validation Accuracy for Best SVM Model:", average_best_svm_cross_val_accuracy)

NameError: name 'GridSearchCV' is not defined

**Best SVM Model Performance Overview:**
The best SVM model demonstrated an impressive accuracy of around 81.9%, highlighting its effectiveness in classifying various topics.

**Detailed Analysis from the Best SVM Model Classification Report:**
1. **High Precision, High Recall:**
   - **Cloud Computing, Contacts, UI/UX:**
     - Precision: 100%, Recall: 100%
   
2. **Balanced Precision and Recall:**
   - **Data Science, DevOps, Mobile Development:**
     - Precision: Varies from 71% to 88%, Recall: Varies from 83% to 98%

3. **Challenges in Precision and Recall:**
   - **Enrollment, Software Engineering:**
     - Precision and Recall: Varies from 40% to 67%

**Insights from the Best SVM Model Confusion Matrix:**
The confusion matrix provides a detailed breakdown of true positives, true negatives, false positives, and false negatives for each class, offering valuable insights into specific challenges faced by the model. Analyzing these patterns can help identify areas for improvement.

**Average Cross-Validation Accuracy for Best SVM Model: 80.4%**
The average cross-validation accuracy of approximately 80.4% further supports the robustness of the model.

**Observations Across Different Classes:**
- Several classes, such as Cloud Computing, Contacts, and UI/UX, showcase both high precision and recall, indicating accurate predictions.
- Data Science, DevOps, and Mobile Development demonstrate a balanced performance, with both precision and recall showing moderate to high values.
- Enrollment and Software Engineering present challenges, with precision and recall varying across a wider range.

**Overall Observation:**
The best SVM model exhibits strong performance across various classes. Precision measures the accuracy of positive predictions, while recall measures the coverage of actual positive instances. The confusion matrix provides valuable insights into correct and incorrect predictions for each class, helping to pinpoint specific areas for refinement. Consideration of hyperparameter tuning or additional data may contribute to enhancing overall performance, particularly for classes with suboptimal precision or recall.

### RandomForestClassifier

In [13]:
# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_model.fit(X_train_vectorized, y_train)

# Predictions on the test set using the Random Forest model
rf_predictions = rf_model.predict(X_test_vectorized)

# Classification report for the Random Forest model
rf_classification_report = classification_report(y_test, rf_predictions)

# Confusion matrix for the Random Forest model
rf_confusion_matrix = confusion_matrix(y_test, rf_predictions)

# Cross-validation with the Random Forest model
rf_cross_val_scores = cross_val_score(rf_model, X_train_vectorized, y_train, cv=5)
average_rf_cross_val_accuracy = np.mean(rf_cross_val_scores)

print("Random Forest Model Classification Report:\n", rf_classification_report)
print("\nRandom Forest Model Confusion Matrix:\n", rf_confusion_matrix)
print("\nAverage Cross-Validation Accuracy for Random Forest Model:", average_rf_cross_val_accuracy)

Random Forest Model Classification Report:
                       precision    recall  f1-score   support

     Cloud Computing       1.00      1.00      1.00         5
            Contacts       1.00      1.00      1.00         1
       Cybersecurity       0.67      0.50      0.57         4
        Data Science       0.82      0.98      0.89        46
              DevOps       0.83      0.83      0.83         6
          Enrollment       1.00      0.40      0.57         5
       Miscellaneous       0.83      0.62      0.71        16
  Mobile Development       0.80      1.00      0.89         8
Software Engineering       0.60      0.30      0.40        10
               UI/UX       0.67      1.00      0.80         4

            accuracy                           0.81       105
           macro avg       0.82      0.76      0.77       105
        weighted avg       0.81      0.81      0.79       105


Random Forest Model Confusion Matrix:
 [[ 5  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  

**Random Forest Model Performance Overview:**
The Random Forest Model showcased a commendable accuracy of around 81%, signifying its effectiveness in classifying diverse topics.

**Insights from the Random Forest Model Classification Report:**
1. **High Precision and Recall:**
   - **Cloud Computing, Contacts, Mobile Development, UI/UX:**
     - Precision: 80% to 100%
     - Recall: 100%
  
2. **Balanced Precision and Recall:**
   - **Data Science, DevOps:**
     - Precision: 82% to 83%
     - Recall: 83% to 98%
  
3. **Challenges in Precision and Recall:**
   - **Cybersecurity, Enrollment, Software Engineering:**
     - Precision and Recall: Varies from 30% to 67%

**Insights from the Random Forest Model Confusion Matrix:**
The confusion matrix provides a detailed breakdown of true positives, true negatives, false positives, and false negatives for each class. This offers valuable insights into specific challenges faced by the model, aiding in pinpointing areas for improvement.

**Average Cross-Validation Accuracy for Random Forest Model: 81.4%**
The average cross-validation accuracy of approximately 81.4% further supports the robustness of the model.

**Observations Across Different Classes:**
- Certain classes, such as Cloud Computing, Contacts, Mobile Development, and UI/UX, demonstrate both high precision and recall, indicating accurate predictions.
- Data Science and DevOps exhibit a balanced performance, with both precision and recall showing moderate to high values.
- Cybersecurity, Enrollment, and Software Engineering present challenges, with precision and recall varying across a range.

**Overall Observation:**
The Random Forest Model exhibits strong performance across various classes. Precision measures the accuracy of positive predictions, while recall measures the coverage of actual positive instances. The confusion matrix provides valuable insights into correct and incorrect predictions for each class, helping to identify areas for refinement. Consideration of hyperparameter tuning or addressing class imbalances may contribute to enhancing overall performance, particularly for classes with suboptimal precision or recall.

# Testing prediction

In [14]:
# Test question
test_question = "How long is the cybersecurity course"

# Tokenize and preprocess the test question
test_tokens = [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(test_question) if word.isalnum() and word not in string.punctuation and word not in stop_words]

# Convert tokens back to a string
cleaned_test_question = ' '.join(test_tokens)

# Vectorize the cleaned test question using the same generic vectorizer
test_question_vectorized = vectorizer.transform([cleaned_test_question])

# Make a prediction using the trained Random Forest model
predicted_tag = rf_model.predict(test_question_vectorized)[0]

print("Predicted Tag:", predicted_tag)

Predicted Tag: Cybersecurity


### GridSearchCV for Random Forest Classifier

In [15]:
# Define the parameter grid to search for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]  # Include valid options for max_features
}

# Create a GridSearchCV object for Random Forest
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5)
grid_search_rf.fit(X_train_vectorized, y_train)

# Get the best hyperparameters for Random Forest
best_params_rf = grid_search_rf.best_params_

# Train the Random Forest model with the best hyperparameters
best_model_rf = RandomForestClassifier(**best_params_rf)
best_model_rf.fit(X_train_vectorized, y_train)

# Predictions on the test set using the best Random Forest model
best_rf_predictions = best_model_rf.predict(X_test_vectorized)

# Classification report for the best Random Forest model
best_rf_classification_report = classification_report(y_test, best_rf_predictions)

# Confusion matrix for the best Random Forest model
best_rf_confusion_matrix = confusion_matrix(y_test, best_rf_predictions)

# Cross-validation with the best Random Forest model
best_rf_cross_val_scores = cross_val_score(best_model_rf, X_train_vectorized, y_train, cv=5)
average_best_rf_cross_val_accuracy = np.mean(best_rf_cross_val_scores)

print("Best Random Forest Model Classification Report:\n", best_rf_classification_report)
print("\nBest Random Forest Model Confusion Matrix:\n", best_rf_confusion_matrix)
print("\nAverage Cross-Validation Accuracy for Best Random Forest Model:", average_best_rf_cross_val_accuracy)

Best Random Forest Model Classification Report:
                       precision    recall  f1-score   support

     Cloud Computing       1.00      1.00      1.00         5
            Contacts       0.00      0.00      0.00         1
       Cybersecurity       1.00      0.50      0.67         4
        Data Science       0.79      0.98      0.87        46
              DevOps       0.83      0.83      0.83         6
          Enrollment       1.00      0.40      0.57         5
       Miscellaneous       0.83      0.62      0.71        16
  Mobile Development       0.89      1.00      0.94         8
Software Engineering       0.62      0.50      0.56        10
               UI/UX       1.00      1.00      1.00         4

            accuracy                           0.82       105
           macro avg       0.80      0.68      0.72       105
        weighted avg       0.82      0.82      0.80       105


Best Random Forest Model Confusion Matrix:
 [[ 5  0  0  0  0  0  0  0  0  0]
 [

In [16]:
# Get the best hyperparameters for Random Forest
best_params_rf

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 50}

Average Cross-Validation Accuracy for Best Random Forest Model: 0.8187033849684452

The best Random Forest model's classification report provides insights into its performance on various classes. Here are some key observations:
- Precision, Recall, and F1-Score: Precision represents the accuracy of positive predictions, recall measures the ability to capture all positive instances, and F1-score balances both. The results vary across classes.
- Class-Specific Insights: Classes like "Contact," "Cybersecurity_Certifications," and "Event" have perfect precision, recall, and F1-scores, indicating accurate predictions. Some classes, such as "Data_Ethics_and_Privacy" and "Data_Science_Admission_Eligibility_Criteria," have low or zero scores, suggesting challenges in prediction.
- Average Metrics: The overall accuracy of the model is 46%, indicating its ability to correctly classify instances. Macro and weighted averages for precision, recall, and F1-score provide a holistic view of the model's performance across all classes.
- Confusion Matrix: The confusion matrix breaks down the actual vs. predicted counts for each class. Diagonal elements represent correct predictions, while off-diagonal elements indicate misclassifications.
- Cross-Validation Accuracy: The average cross-validation accuracy is around 44%, showing consistency in performance across different data splits. Improvements: While the model performs well in some classes, there is room for improvement in others. Further optimization or exploration of data features may enhance overall performance.

### XGBoost model

In [17]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [18]:
# Create an XGBoost model
xgb_model = XGBClassifier()

# Train the XGBoost model
xgb_model.fit(X_train_vectorized, y_train)

# Predictions on the test set using the XGBoost model
xgb_predictions = xgb_model.predict(X_test_vectorized)

# Classification report for the XGBoost model
xgb_classification_report = classification_report(y_test, xgb_predictions)

# Confusion matrix for the XGBoost model
xgb_confusion_matrix = confusion_matrix(y_test, xgb_predictions)

# Accuracy for the XGBoost model
xgb_accuracy = accuracy_score(y_test, xgb_predictions)

print("XGBoost Model Classification Report:\n", xgb_classification_report)
print("\nXGBoost Model Confusion Matrix:\n", xgb_confusion_matrix)
print("\nAccuracy for XGBoost Model:", xgb_accuracy)

XGBoost Model Classification Report:
                       precision    recall  f1-score   support

     Cloud Computing       1.00      1.00      1.00         5
            Contacts       0.33      1.00      0.50         1
       Cybersecurity       1.00      0.75      0.86         4
        Data Science       0.88      0.91      0.89        46
              DevOps       1.00      0.83      0.91         6
          Enrollment       0.33      0.40      0.36         5
       Miscellaneous       0.82      0.56      0.67        16
  Mobile Development       0.89      1.00      0.94         8
Software Engineering       0.44      0.40      0.42        10
               UI/UX       0.67      1.00      0.80         4

            accuracy                           0.79       105
           macro avg       0.74      0.79      0.74       105
        weighted avg       0.81      0.79      0.79       105


XGBoost Model Confusion Matrix:
 [[ 5  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  

Accuracy for XGBoost Model: 0.8285714285714286

The XGBoost model achieved an accuracy of 82.86%. Here are some key performance metrics for specific classes:

**High Performance (Precision, Recall, F1-Score):**
- 'Event': 100% precision, recall, and F1-score.
- 'Mpesa_Mini_Apps_API_Course_Information', 'aws_cloud_course', 'aws_cloud_practitioner', 'cloud_computing', 'cloud_developer_role', 'Software_Engineering_Mobile_Track', 'ai_integration': All achieved perfect scores.

**Low Performance (Precision, Recall, F1-Score):**
- Several classes, such as 'Data_Science_Career', 'Data_Science_Courses', 'Data_Science_Industry_Relevance', 'Data_Science_Networking_and_Collaboration_Opportunities', and 'learning_resources', showed low scores, indicating room for improvement.

**Overall Observation:**
The model struggles with certain classes, potentially requiring further tuning or additional data for those categories.

# Testing XGBoost Model Prediction

In [19]:
# Test question
test_question = "How many students are in Moringa school"

# Tokenize and preprocess the test question
test_tokens = [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(test_question) if word.isalnum() and word not in string.punctuation and word not in stop_words]

# Convert tokens back to a string
cleaned_test_question = ' '.join(test_tokens)

# Vectorize the cleaned test question using the same generic vectorizer
test_question_vectorized = vectorizer.transform([cleaned_test_question])

# Make a prediction using the trained XGBoost model
predicted_tag = xgb_model.predict(test_question_vectorized)[0]

print("Predicted Tag:", predicted_tag)

Predicted Tag: Miscellaneous


# Conclusion

In summary:

1. **Multinomial Naive Bayes Model:**
   - Achieved an accuracy of 66.67%.
   - Classes showed varying precision and recall values, highlighting potential areas for improvement.
   - Precision measures positive prediction accuracy, while recall indicates coverage of actual positive instances.
   - Adjustments may benefit classes with low precision or recall.

2. **GridSearchCV Multinomial Naive Bayes:**
   - Achieved an overall accuracy of 73.33%.
   - Notable variations in precision and recall across different classes.
   - Specific classes exhibited high precision and low recall, emphasizing the need for adjustments.
   - The confusion matrix aids in identifying correct and incorrect predictions for each class.

3. **TF-IDF Vectorizer and SVM Classifier:**
   - Demonstrated an outstanding accuracy of 81.9%.
   - Varied precision and recall values across classes.
   - Confusion matrix insights help understand correct and incorrect predictions.
   - Consideration of hyperparameter tuning or additional data may enhance performance.

4. **Cross-validation with TF-IDF + SVM:**
   - Achieved a solid average cross-validation accuracy of approximately 80.2%.
   - High precision and moderate recall in some classes.
   - Challenges observed in precision and recall for certain classes.
   - Confusion matrix insights provide details on true positives, true negatives, false positives, and false negatives.

5. **GridSearchCV for SVM:**
   - Best SVM model achieved an accuracy of 81.9%.
   - Varied precision and recall across classes.
   - Classes like Cloud Computing, Contacts, and UI/UX showed high precision and recall.
   - Consideration of hyperparameter tuning may further enhance performance.

6. **Random Forest Classifier:**
   - Achieved a commendable accuracy of around 81%.
   - High precision and recall in certain classes.
   - Challenges observed in precision and recall for Cybersecurity, Enrollment, and Software Engineering.
   - Consideration of hyperparameter tuning or addressing class imbalances may improve performance.

7. **GridSearchCV for Random Forest Classifier:**
   - Best Random Forest model achieved an average cross-validation accuracy of 81.87%.
   - Class-specific insights provided, indicating areas for improvement.
   - Confusion matrix breakdown aids in identifying correct and incorrect predictions.

8. **XGBoost Model:**
   - Achieved an accuracy of 83.86%.
   - Varied performance across classes.
   - Some classes showed high precision, recall, and F1-score, while others indicated room for improvement.
   - Further tuning or additional data may be beneficial for certain categories.

Overall, the evaluation of different models and techniques provides insights into their strengths and areas for refinement. Consideration of hyperparameter tuning, adjustments to class-specific challenges, and iterative model refinement are suggested for enhancing overall performance in text classification tasks.